In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
from tabulate import tabulate
import os
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support.ui import Select
import requests

In [3]:
ticker_df = pd.read_csv('constituents_csv.csv')

In [4]:
ticker_list = list(ticker_df['Symbol'])

In [ ]:
list_news = []
site = 'http://stockwatch.com'
for number in range(1, len(ticker_list)):
    profile = FirefoxProfile()
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk", 'application/zip, application/x-zip')
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.dir", "C:/Users/zhav1k/Documents/Python Scripts/diploma/custom_data/stockwatch")
    driver = webdriver.Firefox(firefox_profile=profile)
    url = 'https://www.stockwatch.com/News/Search.aspx'
    driver.implicitly_wait(10)
    driver.get(url)

    
    login = driver.find_element_by_xpath(("//input[@id='PowerUserName'][@name='ctl00$PowerUserName']"))
    login.send_keys("zhav1k")
    pss = driver.find_element_by_xpath(("//input[@id='PowerPassword'][@name='ctl00$PowerPassword']"))
    pss.send_keys('1008985')
    lgn = driver.find_element_by_xpath(("//input[@id='Login'][@class='sprite login_button']"))
    lgn.click()

    login = driver.find_element_by_xpath(("//input[@id='PowerUserName'][@name='ctl00$PowerUserName']"))
    login.send_keys("zhav1k")
    pss = driver.find_element_by_xpath(("//input[@id='PowerPassword'][@name='ctl00$PowerPassword']"))
    pss.send_keys('1008985')
    lgn = driver.find_element_by_xpath(("//input[@id='Login'][@class='sprite login_button']"))
    lgn.click()
    
    ticker = driver.find_element_by_xpath(("//input[@id='MainContent_tSymbol'][@name='ctl00$MainContent$tSymbol']"))
    ticker.send_keys(str(ticker_list[number]))
    date = driver.find_element_by_name('ctl00$MainContent$tSymbolFrom')
    date.clear()
    date.send_keys('20090101')

    all_us = Select(driver.find_element_by_xpath("//select[@id='MainContent_dSymbolFeed'][@name = 'ctl00$MainContent$dSymbolFeed']"))

    all_us.select_by_value('U')

    button = driver.find_element_by_xpath(("//input[@id='MainContent_bSymbol'][@class='sprite submit_button']"))
    button.click()
    
    soup_frst =BeautifulSoup(driver.page_source)
    a = soup_frst.find('tr', { "class" : ['gridViewPager']})
    num_pages = int(a.findAll('a')[-1].text)
    
    
    for page in range(1, num_pages + 1):
        if page == 1:
            soup =BeautifulSoup(driver.page_source)
            for link in soup.findAll('tr', { "class" : ["gridViewRow", 'gridViewAltRow']}):
                r = requests.get(site + link.findAll('a')[1]['href'] )
                soup_test = BeautifulSoup(r.text)
                art = ""
                try:
                    for txt in soup_test.find('span', {'id': 'MainContent_NewsText'}):    
                        try:
                            art+= txt.text
                        except AttributeError:
                            continue
                except TypeError:
                    continue
                list_news.append({'Date': link.find('td').text, 'ticker': link.find('a').text, 'title': link.findAll('a')[1].text, 'article': art})
        else:
            driver.find_element_by_link_text(str(page)).click()
            soup =BeautifulSoup(driver.page_source)
            for link in soup.findAll('tr', { "class" : ["gridViewRow", 'gridViewAltRow']}):
                r = requests.get(site + link.findAll('a')[1]['href'] )
                soup_test = BeautifulSoup(r.text)
                art = ""
                try:
                    
                    for txt in soup_test.find('span', {'id': 'MainContent_NewsText'}):    
                        try:
                            art+= txt.text
                        except AttributeError:
                            continue
                except TypeError:
                    continue
                list_news.append({'Date': link.find('td').text, 'ticker': link.find('a').text, 'title': link.findAll('a')[1].text, 'article': art})
    
    df = pd.DataFrame(list_news)
    df.to_csv('ticker_' + str(number) + '.csv', index = False)
    list_news = []
    del(df)
    driver.quit()

C:\Users\zhav1k\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\zhav1k\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [7]:
df = pd.DataFrame(list_news)